STA 663 Final Project
===
Final Report
---


Section 1:  Basic Information
---
Group members:  Hanyu Song, Azeem Zaman

Paper:  Persistent homology transform for modeling shapes and surfaces

Authors:  Turner, Katharine; Mukherjee, Sayan; Boyer, Doug



Section 2:  Project Abstract
---
### Abstract
Our goal was to develop a package to implement the results of the paper by Turner et. al. In particular, we wrote codes to compute the persistent homology transform (PHT) of an object in $\mathbb{R}^3$ and shapes in $\mathbb{R}^2$. PHT is a statistic that completely describes a shape or surface and allows us to determine a metric on the space of piecewise linear shapes, thereby possibly useful for statistical analysis such as clustering. 

### Background

The paper introduces a tool that can be used to perform statistical shape analysis on objects in $\mathbb{R}^3$ and shapes in $\mathbb{R}^2$.  The result can be of interest to topological data analysists (TDA), researchers modeling shapes (such as medical imaging) and morphologists. One of the paper authors use this to compute the distance between heel bones in primates to generate a tree, which can be compared with a tree generated from the genetic distances between primate species.  

Section 3:  Code
---
This section contains a general description of each function, including:
1.  A function to read in files containing the data
2.  A function to construct a persistence diagram given a direction
3.  A function to calculate the distance between persistence diagrams
4.  Functions to generate directions for the construction of persistence diagrams

### a. Modules requirement

The following packages are required for implementation: `math`, `multiprocessing`, `numpy`, `scipy`, `glob` and `numba`.


In [5]:
import math
import multiprocessing as mp
import numpy as np
import scipy.io as sio
import glob
from scipy.optimize import linear_sum_assignment
from numba import jit


### b. Functions for reading in Shapes

Two functions for reading in data are included in the package. The first `read_file` is for reading in text files saved with raw shape data; the second `read_closed_shape` is used to read Matlab `.mat` files saved with closed shape data. Note that each file contains the data of only one shape. Both functions can read all relevant files in a specified directory; both return a list of vertices and edges of each shape, with the vertices and edges saved in two separate `numpy.ndarray`'s.

The usage of each function are explained in further details below:


####   (1)  `read_file(list_files, d)` : Reads in raw shape data files. 

##### Parameters:

`list_files`: A list of text file names. Each file is saved with the raw shape data from one shape. 

`d`: The dimension of the shape, either 2 or 3. 

a. Note that a single dimension parameter is required because we will only compute distances between shapes with the same dimension. It does not make sense to compare objects in $\mathbb{R}^3$ and shapes in $\mathbb{R}^2$.

b. Text files are required to be structured as follows:
1.  The first line should contain two numbers. The first number is the number of vertices in the shape, and the second is the number of edges.
2.  The next lines contain the coordinates of the vertices, one per line. The points should be seperated by spaces.
3.  The last set of lines should contain two integers, representing vertices that have an edge in between.

An example file is given below:


    4 4 <- Number of vertices, number of edges
    -1 1 <- vertex 1
    1 1
    1 -1
    -1 -1 <- vertex 4
    1 2 <- edge from 1 to 2
    2 3
    3 4
    4 1 <- edge from 4 to 1

##### Returns:

`list_objects`: A list of lists. Each embedded list contains two `numpy.ndarray`'s: the first array contains coordinates of the vertices of one shape; the second contains the location of the edges of the shape. 

##### Function `read_files`:

In [6]:
def read_files(list_files, d):
	"""
    This function reads in text files saved with raw shape data, and 
    outputs information vertices and edges of each shape in a list.
       
    Args:
        list_files (list): a list of text file names. Each file is 
                            saved with the raw shape data of one shape.
        d (int): The dimension of the shape, either 2 or 3. 
    Returns:
        list_objects: a list of lists of numpy.ndarray, i.e.Each 
                        embedded list contains two `numpy.ndarray`'s:
                        the first array contains coordinates of all the 
                        vertices in one shape; the second contains 
                        the location of the edges in the shape 
                        (e.g., array([1,2]) means there exists an edge 
                        between vertex 1 and 2).
	"""

	list_objects = []
	for cur_file in list_files:
		with open(cur_file, "r") as f:
			line = f.readline()
			splitline = line.split()
			num_vert = int(splitline[0])
			num_edges = int(splitline[1])

			vertices = np.empty((num_vert, d))
			edges = np.empty((num_edges, 2))

			# dictionary of vertices {i: v_i}

			for i in range(num_vert):
				line = f.readline()
				splitline = line.split()
				numeric_line = [float(x) for x in splitline]
				vertices[i,:] = np.array(numeric_line)
			for i in range(num_edges):
				line = f.readline()
				splitline = line.split()
				numeric_line = [float(x) for x in splitline]
				edges[i,:] = np.array(numeric_line)
			list_objects.append([vertices, edges])
	return(list_objects)

##### Example:

An example of implementation is given below. Two text file names `'test_obj','test_obj2'` are included in the `list_files`. Each file contains data of shape in $\mathbb{R}^2$, hence $d = 2$. 

In [7]:
res = read_files(list_files = ['test_obj','test_obj2'],d = 2)
res

[[array([[-1.,  1.],
         [-1., -1.],
         [ 1.,  1.],
         [ 1., -1.]]), array([[ 1.,  2.],
         [ 1.,  3.],
         [ 3.,  4.],
         [ 2.,  4.]])], [array([[ 0., -1.],
         [ 0.,  0.],
         [ 1.,  1.],
         [ 2.,  0.],
         [ 3.,  0.],
         [ 3.,  1.],
         [ 2., -1.]]), array([[ 1.,  2.],
         [ 2.,  3.],
         [ 3.,  4.],
         [ 4.,  5.],
         [ 5.,  6.],
         [ 7.,  5.]])]]

As can be seen, the function returns a list of two lists. The first embedded list contains two `numpy.ndarray`'s. The first array 

`array([[-1.,  1.],
        [-1., -1.],
        [ 1.,  1.],
        [ 1., -1.]])` 
        
contains the coordinates of vertices of the shape from the first file `text_obj`.

The second array `array([[ 1.,  2.]` is the location of the edges in the shape, namely an edge exists between vertex 1 and 2.

#### (2)  `read_closed_shape(directory) `: Reads in data of closed shapes save in `.mat` format. 

This function assumes that the shapes are closed, by which we mean that each vertex it connected to the next vertex (i.e.vertex $n$ is connected to vertex $n+1$) and the last vertex is connected to the first.  This is a very specific function, but also a common format used in image analysis.  

##### Parameters:

`directory`: Path to the directory where all the relevant `.mat` files are saved.

##### Returns:
`shapes`: A list of lists. Each embedded list contains two `numpy.ndarray`'s: the first array contains the coordinates of the vertices of one shape; the second contains the location of the edges of the shape.

##### Function  `read_closed_shape`:

In [7]:
def read_closed_shapes(directory):
	"""
	This function reads in all .mat files a specified directory. 
    It assumes that the shapes are closed, by which we mean that 
    each vertex it connected to the next vertex (i.e.vertex $n$ 
    is connected to vertex $n+1$) and the last vertex is connected 
    to the first.  
    
    Args: 
        directory (string): the path to the directory. When referring
        to the current directory, simply use "./".
    Returns:
        shapes (list): A list of lists. Each embedded list contains 
        two `numpy.ndarray`'s: the first array contains the coordinates 
        of the vertices in one shape; the second contains the location 
        of the edges in the shape.
        
	"""
	query = directory + "*.mat"
	files = glob.glob(query)
	shapes = []
	for file in files:
		vertices = sio.loadmat(file)['x']
		N = vertices.shape[0]
		edges = np.zeros((N,2))
		edges[N-1,:] = np.array([N, 1])
		for i in range(N-1):
			edges[i,:] = np.array([i+1, i+2])
		shapes.append([vertices, edges])
	return shapes

##### Example:

The example below demonstrates reading in all the `.mat` files in the current directory. As can be seen, the function returns a list of one list. The embedded list contains two `numpy.ndarray`'s. The first array contains the vertices coordinates of the shape from file `Class1_Sample1.mat`. The second array is the location of the edges in the shape, e.g., an edge exists between vertex 1 and 2, vertex 2 and 3. (Indeed this is a closed shape, so vertex $n$ is connected to vertex $n+1$, for all $n$)

In [9]:
res_closed_shp = read_closed_shapes('./')

In [11]:
print(res_closed_shp[0][0])

[[  2 101]
 [  3 100]
 [  4 100]
 [  5  99]
 [  6  98]
 [  7  98]
 [  8  98]
 [  9  97]
 [  9  96]
 [ 10  95]
 [ 10  94]
 [ 10  93]
 [ 10  92]
 [ 10  91]
 [ 10  90]
 [ 10  89]
 [ 10  88]
 [ 10  87]
 [ 10  86]
 [ 11  85]
 [ 11  84]
 [ 11  83]
 [ 11  82]
 [ 11  81]
 [ 11  80]
 [ 11  79]
 [ 11  78]
 [ 11  77]
 [ 11  76]
 [ 12  75]
 [ 13  76]
 [ 14  75]
 [ 15  74]
 [ 14  73]
 [ 13  72]
 [ 14  71]
 [ 15  70]
 [ 16  69]
 [ 17  68]
 [ 18  67]
 [ 19  66]
 [ 19  65]
 [ 19  64]
 [ 20  64]
 [ 21  64]
 [ 22  63]
 [ 23  62]
 [ 23  61]
 [ 24  60]
 [ 24  59]
 [ 25  58]
 [ 26  58]
 [ 27  58]
 [ 28  57]
 [ 29  57]
 [ 30  56]
 [ 30  55]
 [ 31  55]
 [ 32  54]
 [ 33  54]
 [ 34  54]
 [ 35  53]
 [ 35  52]
 [ 34  51]
 [ 35  51]
 [ 36  51]
 [ 37  50]
 [ 38  49]
 [ 39  49]
 [ 40  49]
 [ 41  48]
 [ 42  47]
 [ 42  46]
 [ 42  45]
 [ 43  45]
 [ 44  45]
 [ 45  44]
 [ 46  43]
 [ 47  42]
 [ 48  41]
 [ 48  40]
 [ 48  39]
 [ 48  38]
 [ 49  37]
 [ 50  36]
 [ 51  35]
 [ 52  34]
 [ 53  33]
 [ 54  33]
 [ 55  32]
 [ 56  31]

### c. Functions for persistence diagram construction 

A function to construct a persistence diagram given a direction is included in the package. The functionality 

A persistence diagram is a filtration.  We start with an object and "build" the object in a certain direction.  We record when each point in the object first appears (is "born") and when it merges into an object that already exists (it "dies).  Consider the shape below:

![title](full_fig.png)

We will construct this figure in the direction $v = (0,1)$.  If we imagine moving upwards across the figure, the first height at which we will see any points of the diagram is $h=-1$.  We see that vertices 1 and 7 are born at $h = -1$, as shown below:

![h1](persist_diag1.png)

The next height at which something something interesting happens is $h = 0$, at which time three more points are born.  All of these points, however, die immediately.  Vertex 2 merges with vertex 1 and vertices 4 and 5 merge with vertex 7.  At this time we have two unconnected components.

![fig2](persist_diag2.png)

Once we reach $h = 1$, we have finished constructing the diagram.  Vertex 6 dies immediately because it merges with vertex 5.  Vertex 3 joints the two components that were previously disjoint.  Since vertices 1 and 7 were both born at $h=-1$, we make a convention that lower numbered vertices will be considered the root and higher numbered vertices will merge with them.  Thus at time $h=1$ vertex 7 dies, as it merges with vertex 1.

![fig3](persist_diag3.png)

A plot of the persistence diagram is given below.  The red point represents vertex 1, which never dies.  We consider the point to be at $(-1,\infty)$.  The line in the figure is the diagonal.  A point on the diagonal is one that is born and dies at the same instant.  

![diagram](diagram.png)

To impliment this process of constructing persistene diagrams, we need to be able to tell when a vertex is born and when the vertex merges with another component.  For a fixed direction $v$ (which we will always take to be unit length), we can easily determien the height of a vertex. The height of a vertex $p$ with respect to a direction $v$ is simply the inner product of these two vectors.  The function that calculates the height is given below.  



In [1]:
def direction_height(obj, direction):
	"""
	This function computes the height of all vertices with
	respect to a specified direction and returns the height 
	of each vertex in a dictionary.

	The function also computes, for each vertex, a list of vertices
	that are lower (with respect to direction) 
    
    Args:
        obj:  An object.  Should be a list of length 2.  The first item in the 
        list is a matrix where each row is a vertex.  The second item in the list
        is a matrix in which each row specifies an (undirected) edge between two 
        vertices (this matrix has two columns, regardless of the dimension)
        
        direction: a numpy row vector of appropriate dimension (2 or 3)
        
    Returns:
        dict_heights:  a dictionary that maps v -> height(v) with respect to direction
        dict_neighbors:  a dictionary that maps v to a list of lower neighbors.  By lower
            neighbors we mean vertices that are connected to v by an edge and have a lower
            height than v.  This is convinient information for constructing persistence diagrams
	"""
	
	# this dictionary will contain the vertices that have
	# a lower height with respect to direction
	dict_neighbors = {}

	# this dictionary will map {v : height(v)} for
	# each vertex v
	num_vert = obj[0].shape[0]
	num_edges = obj[1].shape[0]
	heights = obj[0].dot(direction.T)
	dict_heights = {i+1: heights[i] for i in range(num_vert)}
	# sort vertices by height	

	dict_neighbors = {v : [] for v in dict_heights.keys()}

	# for each vertex we find all other vertices that lower
	# height with respect to direction 
	for i in range(num_edges):
		edge = obj[1][i,:]
		if dict_heights[edge[0]] > dict_heights[edge[1]]:
			# if first vertex is higher than second
			# add second vertex to list for first
			# vertex
			dict_neighbors[edge[0]].append(int(edge[1]))

		elif dict_heights[edge[0]] == dict_heights[edge[1]]:
			# if they have the same height, we add the 
			# lower index to the list of the larger index
			if edge[0] > edge[1]:
				dict_neighbors[edge[0]].append(int(edge[1]))
			# cannot have edge from vertex to itself
			# so if above condition is false, we must
			# have edge[1] > edge[0]
			else:
				dict_neighbors[edge[1]].append(int(edge[0]))
		# if other conditions are not met, then edge[1] is
		# higher than edge[0]
		else:
			dict_neighbors[edge[1]].append(int(edge[0]))
	return dict_heights, dict_neighbors


The harder part of the algorithm is determining when components merge.  To keep track of this, we will use a modified version of the Union-Find algorithm.  We define a class called `Tree`.  A `Tree` object has a parent and every tree begins with its own parent.  For convenience, it also has a name.  We will name a tree after a vertex.  So for each vertex $i$ there will be a tree such that `Tree.name = i`.  

The function `Find` finds the root of a tree by travelling up the tree until we find a tree that is equal to its parent.  The function `height_Union` joins two trees in a specific way.  If the two input trees have the same root, nothing is done.  If the two trees have different heights, we make the root of one tree equal to the root of the other tree, effectively merging the trees.  As persistence diagrams are built upwards, we make the root of the joint tree the vertex that has the smaller height.  If the vertices have the same height, we make the root the vertex with the lower number.  

These functions are used internally and will not be called by a user.  

In [3]:
class Tree:
	def __init__(self, name):
		self.parent = self
		self.name = name
		self.rank = 0
	
def Find(x):
	"""
	This function determines the root of the tree
	that x is in.  It works recursively. x should
	be an object of class Tree.
	"""
	if x.parent != x:
		x.parent = Find(x.parent)	
	return x.parent

def height_Union(x, y, dict_heights):
	"""
	This function takes the union of two nodes.
	It does this by changing the root one tree
	to be the root of the other tree.  It changes the
	root based on height.  The root becomes the node with
	the lowest height.  So the node that is born first
	becomes the root.  

	If the two roots have the same height, the lowest 
	number becomes the root.  For example, if we have vertex
	1 and vertix 3 at the same height, vertex 1 will become
	the root.  

	Inputs:
		x,y:  objects of Tree class
		dict_heights:  a map v-> h, where v is a vertex,
		which should be the .name of some tree and h is
		the height with respect to some direction
        
    Returns:
        No returns.  Tree objects are merged.
	"""
	x_root = Find(x)
	y_root = Find(y)
	if x_root == y_root:
		return None
	if dict_heights[x_root.name] < dict_heights[y_root.name]:
		y_root.parent = x_root
	elif dict_heights[x_root.name] == dict_heights[y_root.name]:
		if x.name < y.name:
			y_root.parent = x_root
		else:
			x_root.parent = y_root
	else:
		x_root.parent = y_root

We have created a class called `Diagram` to store persistence diagrams.  It has a list of finte points, which will be of the form $(\text{birth,death})$ and infinite points, which are stored as the birth time.  We have methods to add points to both of these lists.

In [4]:
class Diagram:
	def __init__(self):
		self.points = []
		self.infpoints = []
	def addpt(self, pt):
		self.points.append(pt)
	def addinfpt(self, pt):
		self.infpoints.append(pt)

The function that uses this code is `make_diagram`, shown below.  This function increases the height in steps.  At each step we keep track of which vertices have died.  We take the vertices that are born at that height and use `Union` to join them with their lower neighbors.  Using this method we can keep track of when various components merge.    
#### (2)  `make_diagram(dict_heights, dict_neighbors) `: Creates a persistence diagram 

This function uses the output of `direction_height` to create a persistence diagram.  It returns an object of class diagram. This function is used internally and will not be called by the end user.

##### Parameters:

`dict_heights`: A dictionary mapping a vertex to its height with respect to some direction.

`dict_neighbors`:  A dictionar mapping each vertex to a list with vertices that are connected to the vertex and at a lower height.

##### Returns:
`diag`: An object of class diagram.  

##### Function  `make_diagram`:

In [10]:
def make_diagram(dict_heights, dict_neighbors):
    """
    This function uses the output of direction_height to create a persistence diagram. 
    It returns an object of class diagram. This function is used internally and will 
    not be called by the end user.
    
    Parameters:
        dict_heights: A dictionary mapping a vertex to its height with respect to some direction.

        dict_neighbors: A dictionar mapping each vertex to a list with vertices that are 
            connected to the vertex and at a lower height.
    Returns:
        diag: An object of class diagram. 
    """
    # create a tree for each vertex
    dict_trees = {v: Tree(v) for v in dict_heights.keys()}
    # get a set of vertices and a set of heights
    vertices = set(dict_heights.keys())
    heights = set(dict_heights.values())
    # sort the heights, will be unique because of set construction
    heights_sorted = sorted(list(heights))
    # for each height, we find the vertices that are alive at that height
    dict_below_h = {h: [] for h in heights}
    for h in heights:
        for v in vertices:
            if dict_heights[v] <= h:
                dict_below_h[h].append(v)
    # dictionary that keeps track of deaths
    # if a vertex has not died, the value will be None
    # if a vertex still has None at the end, it never dies
    dict_deaths = {v: None for v in dict_heights.keys()}
    # set to hold dead vertices
    dead = set()
    # if it has a lower neightbor, it dies immediatly
    # for all such vertices, we can set the time of death
    # to be the time of birth
    for v in vertices:
        # if it has a lower neightbor
        if len(dict_neighbors[v]) != 0:
            # it dies at the time it was born
            dict_deaths.update({v: dict_heights[v]})
            # add it to set of dead vertices
            dead.add(v)
    # the vertices that are alive are those that do not die instantly
    # these are the only onces we need to consider
    alive = vertices.difference(dead)
    # now we increase the height and merge components as we go
    for h in heights_sorted:
        # keep track of which vertices die at this height
        died = set()
        # merge all vertices currently alive with lower vertices
        for v in dict_below_h[h]:
            for neigh in dict_neighbors[v]:
                height_Union(dict_trees[v], dict_trees[neigh],
                        dict_heights)
        # now we consider all vertices that are still alive
        for v in alive:
            # check if v is alive (born at height less than h)
            if dict_heights[v] <= h:
                # boolean:  is v not its own root?
                # if it is not its own root, then it must have merged 
                # with some other class
                own_root = v != Find(dict_trees[v]).name
                if own_root:
                    # has merged with another component
                    # record time of death and mark as dead
                    died.add(v)
                    dict_deaths.update({v: h})
        # update set of vertices that are alive
        alive = alive.difference(died)
    # create an empty diagram
    diag = Diagram()
    # consider each vertex v
    for v in dict_heights.keys():
        # if time of death is none, add a point at infinity
        if dict_deaths[v] == None:
            diag.addinfpt(dict_heights[v])
        else:
            # only add point if not on the diagonal
            # this is designed to reduce the complexity
            # of the problem for the Munkres algorithm
            if dict_heights[v] != dict_deaths[v]:
                diag.addpt((dict_heights[v], dict_deaths[v]))

    return diag

Once we have a diagram, we can consider the distance between diagrams.  If $X$ and $Y$ are diagrams, we can consider bijections $\phi$ between the points and copies of the diagonal in $X$ with the points and copies of the diagonal in $Y$.  Bijections always exist because there are enought copies of the diagonal to create one.  We define the distance between $X$ and $Y$ to be
\begin{align}
dist_p(X,Y) &= \left(\inf_{\phi: X \to Y}\sum_{x \in X} ||x - \phi(x)||_p^p\right)^{1/p}.
\end{align}
The paper suggests that $p=1$ performs best in practice (the $L1$ norm).  The existence of an optimal bijection is proved in the literature.  An example of an optimal bijection is shown below (taken from www.math.uiuc.edu).  The points of $X$ are shown in red and the points of $Y$ are shown in blue.  The red points are paired with nearby blue points.  Most blue points are paired with their projection onto the diagonal.  In practice, we see that we add the projection onto the diagonal for each point.  This is an optimization problem that can be solved using the Hungarian (Munkres) algorithm.  We can calculate the distance between the points in $X$ and the points in $Y$ and the distance from the points in $X$ to the diagonal.  This can be represented as a cost matrix.  

As we will discuss in detail later, this is the bottleneck in our code.  The algorithm is somewhat difficult to implement, so we used an existing implemntation.  Another difficulty is that the first implementation we tried got stuck in an infinite loop.  We settled on using a function from `scipy.optimize`, `linear_sum_assignment`.  This is the distance between the finite points.  We alsoneed to consider the points with one component at infinity.  We will pair these amongst themselves and simply consider the distiance between the birth times.  If two diagrams have a different number of points at infinity, then we would have to pair a point at infinity with a finite point or a point on the diagonal, which is infinitely far away.  The distance between such diagrams is said to be infinite.  

![diagdist](diagram_distance.jpeg)

First we have the code that calculates the distance between the points at infinity.  

####  `inf_pt_dist(diagram1, diagram2, q) `: Calculates the distance between essential classes 

This function computes the distance between the points that correspond to essential classes, i.e. classes where the second component is infinite.  The function assumes that the sets of points are ordered by the first component (the finite one).  If the functions have a different number of such points, the function will print a warning message.

##### Parameters:

`diagram1`: the first persistence diagram

`diagram2`: the second persistence diagram

`q`: we use $L_q$ distance

##### Returns:
`distance`: the distance between the essential classes.  

##### Function  `inf_pt_dist`:

In [12]:
def inf_pt_dist(diagram1, diagram2, q):
    """
	This function computes the distance between the 
	points that correspond to essential classes,
	i.e. classes where the second component is infinite

	This function assumes that the sets of points are
	ordered by the first component (the finite one).
	This should be the case, as points are added to 
	the diagram in this order during construction.

	If the the diagrams have a different number of points
	at infinity, the distance between them is 
	infinite.

	Args:
		diagram1:  first persistence diagram
		diagram2:  second persistence diagram
		q:  we use L_q distance

	Return:  distance between points at infinity
		for the two diagrams

		"""

    n = len(diagram1.infpoints)
    m = len(diagram2.infpoints)
    number = min(m,n)

    if m != n:
        print("Warning:  different number of points are infinity")

    distance = 0
    for i in range(number):
        distance += pow(abs(diagram1.infpoints[i] - 
            diagram2.infpoints[i]), q)
    return distance

We also have two simple helper functions, which are unworthy of comment.  

In [14]:
def L1dist(x,y):
	"""
	This function computes the L1 distance between
	two vectors x and y
	"""
	c = abs(x[0]-y[0]) + abs(x[1] - y[1])
	return c

def diag_len(x):
	"""
	This function calculates the distance from the point
	x (in the persistence diagram) to the diagonal.  This
	function uses the L1 distance and will be used in later 
	functions.
	"""
	return x[1] - x[0]

These two functions are used to calculate the distance matrix used to calculate the distance between points.  The distance matrix is not unexpected, except for the addition of the points on the diagonal.  This makes the matrix square.  

####  `make_dist_mat(diagram1, diagram2, q) `: Makes distance matrix between points of two diagrams 

This function constructs a distance between the points of two diagrams, including points on the diagonal.  It returns this distance matrix.

##### Parameters:

`diagram1`: the first persistence diagram

`diagram2`: the second persistence diagram

`q`: we use $L_q$ distance

##### Returns:
`dist_mat`: the distances between the points of the diagram.  

##### Function  `make_dist_mat`:

In [15]:
def make_dist_mat(diagram1, diagram2, q):
	"""
    This function constructs a distance between the points of two diagrams, including points
    on the diagonal.  It returns this distance matrix.
    """
	n = len(diagram1.points)
	m = len(diagram2.points)
	dist_mat = np.zeros((n+m,n+m))
	for i in range(n):
		for j in range(m):
			# distance from point i in diagram 1 to joint j in diagram 2

			distance = pow(L1dist(diagram1.points[i], diagram2.points[j]),q)
			dist_mat[i, j] += distance
		# distance from point i of diagram 1 to diagonal
		dist_to_diag = pow(diag_len(diagram1.points[i]), q)
		dist_mat[i, m:][np.newaxis, :] += dist_to_diag*np.ones((1,n))
		# now we consider the m copies of the diagonal
	for j in range(m):
		# distiance from jiagonal to point j of diagram 2
		dist_to_diag = pow(diag_len(diagram2.points[j]),q)
		dist_mat[n:,j][:,np.newaxis] += dist_to_diag*np.ones((m,1))
	return dist_mat

Now we have the most computationally intensive function in the code.  This function solves the assignment problem for the finite points of two diagrams.  It uses `linear_sum_assignment` from the python code.

#### `finite_pt_dist(diagram1, diagram2, q) `: Calculates the distance between non-essential classes 

This function computes the smallest distance between the finite points of two persistence diagrams.  This requires matching each point in the first diagram to a point in the second diagram or a point on the diagonal such that the distance between the pairs of points is minimized.  The solution to this problem involves the Munkres (or Hungarian) algorithm, which has already been implemented in Scipy.

##### Parameters:

`diagram1`: the first persistence diagram

`diagram2`: the second persistence diagram

`q`: we use $L_q$ distance

##### Returns:
`distance`: the distance between the non-essential classes.  

##### Function  `finite_pt_dist`:

In [17]:
def finite_pt_dist(diagram1, diagram2, q):
	"""
	This function computes the smallest distance between the
	finite points of two persistence diagrams.  This requires
	matching each point in the first diagram to a point in the 
	second diagram or a point on the diagonal such that the distance
	between the pairs of points is minimized.

	The solution to this problem involves the Munkres (or Hungarian)
	algorithm, which has already been implemented in Python.

	This function returns the distanc between the two diagrams
    
	Args:
		diagram1:  first persistence diagram
		diagram2:  second persistence diagram
		q:  we use L_q distance

	Return:  distance between points at infinity
		for the two diagrams
	"""

	n = len(diagram1.points)
	m = len(diagram2.points)

	# if there are no points, the distance is zero
	if n + m == 0:
		return 0
	else:
		dist_mat = make_dist_mat(diagram1, diagram2, q)
		# now we can compute the total distance
		row_ind, col_ind = linear_sum_assignment(dist_mat)
		total_dist = 0
		for i in range(len(row_ind)):
			row = row_ind[i]
			col = col_ind[i]
			value = dist_mat[row][col]
			total_dist += value
		return total_dist



Now that we can calculate the distance between two persistence diagrams, we are ready to compute the distance between two objects in $\mathbb{R}^2$ or $\mathbb{R}^3$.  For the objects we are considering, the distance (which is indeed a distance based on the results in the paper) is
\begin{align}
dist(M_1,M_2) := \int_{S^{d-1}} dist(X(M_1,v),X(M_2,v))\,dv,
\end{align}
where $X(M_i,v)$ is the persistence diagram for object $M_i$ in direction $v$.  Stability results from the paper reassure that the error in approximating the distance using only a finite sampling of directions should be small.  Thus we approximate this integral by using a sample of points from the circle (in $\mathbb{R}^2$) or the sphere (in $\mathbb{R}^3$).  

If is often useful to consider objects modulo a group of transformations, typically scaling, rotation, and translation.  Descriptions of how to implement these transformations can be found in the paper.  The code that centers and scales the objects is given below.  Rotation is slightly different.  To calculate the distance modulo rotation, we must consider all possible rotations of the second object and select the one that minimizes the distance.  In practice, we select some finite set of rotations to consider.  Furthermore, we do not actually rotate the object.  Instead, we note that the per

####  `center_scale(list_objects, matrix_dir) `: Centers and scales objects with respect to directions 

This function centers and scales the objects in `list_objects` with respect to the directions in `matrix_dir`.

##### Parameters:

`list_objects`: A list of lists. Each embedded list contains two `numpy.ndarray`'s: the first array contains the coordinates of the vertices of one shape; the second contains the location of the edges of the shape.  This is the format returned by the functions that read in shape files.

`matrix_dir`: a matrix where each direction (on the cirlce or sphere) is a column vector.

##### Returns:
`list_objects`: a list of objects of the same format as the input, but scaled and centered 

##### Function  `center_scale`:

In [20]:
def center_scale(list_objects, matrix_dir):
	"""
    This function centers and scales the objects in list_objects with respect to the
    directions in matrix_dir
    
    Args:
        list_objects:  a list of objects, specified as pairs of matrices
        matrix_dir:  a matrix of directions, each unit vector is a column
        
    Returns:
        A list of scaled and centered objects
    """
	k = matrix_dir.shape[1]
	# calculate scaling constant
	N = len(list_objects)
	K = 0
	for i in range(k):
		K += np.cos(2*np.pi*i/k)**2
	for i in range(N):
		obj = list_objects[i][0]
		prod = obj.dot(matrix_dir)
		# minimum of each column is lambda_i
		lambdas = prod.min(axis = 0)
		u = (1/K)*matrix_dir.dot(lambdas)
		list_objects[i][0] = obj - u[np.newaxis, :]
		# now we scale each object
		L = -lambdas.sum()
		list_objects[i][0] /= L
	return list_objects

For this we will need directions.  These simple functions generate those directions, either uniformly or randomly.

In [21]:
# these functions are used to sample directions
# the directions are used to construct persistence diagrams
# we need to construct persistence diagrams in many directions
# the 2-D functions will return exactly n directions, either
# randomly generated or evenly spaced
# the 3-d functions will return at least n directions
def sample_circle(n):
	"""
	This function gives evenly spaced directions
	on the cirlce and returns them in a 2 by n 
	matrix.  Each row represents a direction
	"""
	thetas = np.linspace(0, 2*np.pi, n)
	xs = np.cos(thetas)
	ys = np.sin(thetas)
	directions = np.array([xs,ys]).T
	return directions

def random_circle(n):
	"""
	This function randomly samples n points on the circle
	"""
	draws = np.random.uniform(low=0, high=2*np.pi, size=n)
	xs = np.cos(draws)
	ys = np.sin(draws)
	directions = np.array([xs,ys]).T
	return directions

def sample_sphere(n):
	"""
	This function returns approximately n points
	on the sphere.  It may return more if n is 
	not a perfect square.  It will always return at 
	least n points.  
	"""
	N = np.sqrt(n) + 1
	thetas = np.linspace(0, np.pi, N)
	phis = np.linspace(0, 2*np.pi, N)
	points = np.meshgrid(thetas, phis)
	xs = np.sin(points[0])*np.sin(points[1])
	ys = np.sin(points[0])*np.cos(points[1])
	zs = np.cos(points[0])
	
	matrix = np.vstack([xs.flatten(), ys.flatten(), zs.flatten()]).T
	return matrix


def random_sphere(n):
	"""
	This function returns approximately n randomly
	select points on the sphere.  It may return more if n is 
	not a perfect square.  It will always return at 
	least n points.  
	"""
	N = np.sqrt(n) + 1
	thetas = np.random.uniform(0, np.pi, N)
	phis = np.random.uniform(0, 2*np.pi, N)
	points = np.meshgrid(thetas, phis)
	xs = np.sin(points[0])*np.sin(points[1])
	ys = np.sin(points[0])*np.cos(points[1])
	zs = np.cos(points[0])
	
	matrix = np.vstack([xs.flatten(), ys.flatten(), zs.flatten()]).T
	return matrix

Now that we have code to scale and center, we write two functions to calculate the distance between objects given all necessary persistence diagrams.  The first does not take into account rotation, the second does.  Taking into account rotation involves considering all possible pairs of diagrams.  This is costly, as it involves many repetitions of the slowest part of the code.  First the code without rotation.

####  `calculate_distance(i, j, k, l_diagrams)`: Calculates distance between two objects

This function calculates the distance between two objects

##### Parameters:

`i`: index of first object in l_diagrams
`j`: index of second object in l_diagrams
`k`: number of directions (number of diagrams for each object)
`l_diagrams`:  a list of lists.  Each list contains `k` diagrams for an object.

##### Returns:
`i,j, actual_dist`:  the inputs `i` and `j` are returned for convience when the code is parallelized. The number `actual_dist` is the distance between the two objects.

##### Function  `calculate_distance`:

In [22]:
def calculate_distance(i, j, k, l_diagrams):
	"""
	This function calculates the distance between objects i and j in l_diagrams

	Args:
		i:  index of first object
		j:  index of second object
		k:  number of directions
		l_diagrams:  a list of diagrams for each object in each of the
			k directions
	
	Return:
		The returns a tuple with 3 number: i, j, and the distance between
		these objects under rotation.  The return of i and j is for convenience
		in parallelization.
	"""
	finite_dist = 0
	infinite_dist = 0
	for c in range(k):
		finite_dist += finite_pt_dist(l_diagrams[i][c],
					l_diagrams[j][c], 1)
		infinite_dist += inf_pt_dist(l_diagrams[i][c],
					l_diagrams[j][c], 1)
	actual_dist = (finite_dist + infinite_dist)/k
	return i, j, actual_dist

We also have a function that calculates the distance modulo rotation. 

####  `calculate_distance_scale(i, j, k, l_diagrams)`: Calculates distance between two objects

This function calculates the distance between two objects modulo rotation.

##### Parameters:

`i`: index of first object in l_diagrams
`j`: index of second object in l_diagrams
`k`: number of directions (number of diagrams for each object)
`l_diagrams`:  a list of lists.  Each list contains `k` diagrams for an object.

##### Returns:
`i,j, actual_dist`:  the inputs `i` and `j` are returned for convience when the code is parallelized. The number `actual_dist` is the distance between the two objects.

##### Function  `calculate_distance_scale`:

In [24]:
def calculate_distance_scale(i, j, k, l_diagrams):
	"""
	This function calculates the distance between i and j modulo rotation.
	To do this, we calculate the distance between i and all rotations of j
	and take the minimal distance.  We do not actually rotate the object.
	Instead, we realize that the persistence diagram of a rotation is
	is persistence diagram of a different direction.  Thus we simply rename the 
	persistence diagrams.  

	Args:
		i:  index of first object
		j:  index of second object
		k:  number of directions
		l_diagrams:  a list of diagrams for each object in each of the
			k directions
	
	Return:
		The returns a tuple with 3 number: i, j, and the distance between
		these objects under rotation.  The return of i and j is for convenience
		in parallelization.
	"""
 	# this list holds the distance between object i and object j under each rotation   
	list_dists = []
	for shift in range(k):
		# this is the current rotation
		# we are considering rotation shift of k
		finite_dist = 0
		infinite_dist = 0
		for cur in range(k):
			# (cur + shift)%k renumbers the diagram
			# each diagram moves forward by shift
			# %k wraps around the end when shift + cur >= k
			finite_dist += finite_pt_dist(l_diagrams[i][cur],
										  l_diagrams[j][(cur + shift)%k],
										  1)
			infinite_dist += inf_pt_dist(l_diagrams[i][cur],
										 l_diagrams[j][(cur + shift)%k],
										 1)
		list_dists.append(finite_dist + infinite_dist)
	# distance is the minimum
	actual_dist = min(list_dists)/k
	return i, j, actual_dist

At this point we are almost ready to implement the desired functions.  The last function we need is one that calculates all diagrams for a set of shapes in a set of directions.  This function is given below. 

####  `make_diagrams(list_objects, matrix_dir) `: Makes all diagrams for objects in specified direction 

This function makes diagrams for all shapes/objects in `list_objects` in all directions in `matrix_dir`.

##### Parameters:

`list_objects`: A list of lists. Each embedded list contains two `numpy.ndarray`'s: the first array contains the coordinates of the vertices of one shape; the second contains the location of the edges of the shape.  This is the format returned by the functions that read in shape files.

`matrix_dir`: a matrix where each direction (on the cirlce or sphere) is a column vector.

##### Returns:
`l_diagrams`: a list diagrams, one for each object in each direction

##### Function  `make_diagrams`:

In [27]:
def make_diagrams(list_objects, matrix_dir):
	"""
    This function makes diagrams for all shapes/objects in list_objects in all
    directions in matrix_dir
    
    Args:
        list_objects:  a list of objects, specified as pairs of matrices
        matrix_dir:  a matrix of directions, each unit vector is a column
        
    Returns:
        A list of diagrams for all objects in all directions
    """
	k = matrix_dir.shape[1]
	N = len(list_objects)
	l_diagrams = []
	for i in range(N):
		shape = list_objects[i]
		shape_diagrams = []
		for j in range(k):
			direction = matrix_dir[:,j].T
			# make the diagram for jth direction
			d_heights, d_n = direction_height(shape, direction)
			shape_diagram = make_diagram(d_heights, d_n)
			shape_diagrams.append(shape_diagram)
		l_diagrams.append(shape_diagrams)
	return l_diagrams

### c. Functions
We are now to the main functions that will be run by the end user.  These functions calculate the distance between as set of objects using an approximation with a specified number of directions.  There are four functions in total.  Two that calculate without any scaling (scaling, rotation, and centering) and two that calculate with scaling.  For each pair, one function has parallel processing implemented.  

####  `distance_unscaled(list_objects, k) `: Make a distance matrix 

This function makes a distance matrix for the objects in `list_objects` using `k` directions in the approximation.

##### Parameters:

`list_objects`: A list of lists. Each embedded list contains two `numpy.ndarray`'s: the first array contains the coordinates of the vertices of one shape; the second contains the location of the edges of the shape.  This is the format returned by the functions that read in shape files.

`k`: number of directions to use in approximation

##### Returns:
`dists`: a distance matrix

##### Function  `distance_unscaled`:

In [28]:
def distance_unscaled(list_objects, k):
	"""
    This function calculates the pairwise distance between all objects in list_objects using k directions
    in the approximation
    
    Args:
        list_objects:  a list of objects, specified as pairs of matrices
        k: number of directions to use in approximation
        
    Returns:
        A distance matrix
    """
	N = len(list_objects)
	
	matrix_dir = sample_circle(k).T
	l_diagrams = make_diagrams(list_objects, matrix_dir)
	dists = np.zeros((N,N))
	for i in range(N):
		for j in range(i+1,N):
			r1, r2, dist = calculate_distance(i,j,k,l_diagrams)
			dists[i,j] = dist
	dists += dists.T
	return dists

A now a version that scales the objects.

####  `distance_scaled(list_objects, k) `: Make a distance matrix 

This function makes a distance matrix for the objects in `list_objects` using `k` directions in the approximation.  The functions scales, centers, and rotates the objects.

##### Parameters:

`list_objects`: A list of lists. Each embedded list contains two `numpy.ndarray`'s: the first array contains the coordinates of the vertices of one shape; the second contains the location of the edges of the shape.  This is the format returned by the functions that read in shape files.

`k`: number of directions to use in approximation

##### Returns:
`dists`: a distance matrix

##### Function  `distance_scaled`:

In [29]:
def distance_scaled(list_objects, k):
	"""
    This function calculates the pairwise distance between all objects in list_objects using k directions
    in the approximation.  It centers, scales, and rotates the objects.
    
    Args:
        list_objects:  a list of objects, specified as pairs of matrices
        k: number of directions to use in approximation
        
    Returns:
        A distance matrix
    """
	N = len(list_objects)
	
	matrix_dir = sample_circle(k).T
	scaled_objects = center_scale(list_objects, matrix_dir)
	l_diagrams = make_diagrams(scaled_objects, matrix_dir)
	dists = np.zeros((N,N))
	for i in range(N):
		for j in range(i+1,N):
			r1, r2, dist = calculate_distance_scale(i, j, k, l_diagrams)
			dists[i,j] += dist
	dists += dists.T
	return dists

We now have the same functions again, but with the ability to run the tasks in parallel.  

####  `distance_unscaled_mc(list_objects, k, workers) `: Make a distance matrix 

This function makes a distance matrix for the objects in `list_objects` using `k` directions in the approximation.

##### Parameters:

`list_objects`: A list of lists. Each embedded list contains two `numpy.ndarray`'s: the first array contains the coordinates of the vertices of one shape; the second contains the location of the edges of the shape.  This is the format returned by the functions that read in shape files.

`k`: number of directions to use in approximation

`workers`:  number of jobs to run in parallel

##### Returns:
`dist_mat`: a distance matrix

##### Function  `distance_unscaled_mc`:

In [31]:
def distance_unscaled_mc(list_objects, k, workers):
	"""
    This function calculates the pairwise distance between all objects in list_objects using k directions
    in the approximation
    
    Args:
        list_objects:  a list of objects, specified as pairs of matrices
        k: number of directions to use in approximation
        workers:  number of parallel jobs to run
        
    Returns:
        A distance matrix
    """
	# number of objects
	N = len(list_objects)
	# number of directions	
	matrix_dir = sample_circle(k).T
	# now we create a list of diagrams
	# each item in the list is a list of
	# diagrams for one object in each direction
	# so it is a list of length N with sublists
	# of size k
	l_diagrams = make_diagrams(list_objects, matrix_dir)
	dist_mat = np.zeros((N,N))
	list_inputs = [(i, j, k, l_diagrams) for i in range(N)
										 for j in range(i+1, N)]
	with mp.Pool(processes=workers) as pool:
		distances = pool.starmap(calculate_distance, list_inputs)
	for tup in distances:
		dist_mat[tup[0],tup[1]] = tup[2]
	dist_mat += dist_mat.T
	return dist_mat

Finally, we have a scaled version that is also parallel.  

####  `distance_scaled_mc(list_objects, k, workers) `: Make a distance matrix 

This function makes a distance matrix for the objects in `list_objects` using `k` directions in the approximation.  It centers, scales, and rotates the objects

##### Parameters:

`list_objects`: A list of lists. Each embedded list contains two `numpy.ndarray`'s: the first array contains the coordinates of the vertices of one shape; the second contains the location of the edges of the shape.  This is the format returned by the functions that read in shape files.

`k`: number of directions to use in approximation

`workers`:  number of jobs to run in parallel

##### Returns:
`dist_mat`: a distance matrix

##### Function  `distance_scaled_mc`:

In [32]:
def distance_scaled_mc(list_objects, k, workers):
	"""
    This function calculates the pairwise distance between all objects in list_objects using k directions
    in the approximation.  It centers, scales, and rotates the objects
    
    Args:
        list_objects:  a list of objects, specified as pairs of matrices
        k: number of directions to use in approximation
        workers:  number of parallel jobs to run
        
    Returns:
        A distance matrix
    """
	# number of objects
	N = len(list_objects)
	# number of directions	
	matrix_dir = sample_circle(k).T
	# compute centering constant
	scaled_objects = center_scale(list_objects, matrix_dir)
	# now we create a list of diagrams
	# each item in the list is a list of
	# diagrams for one object in each direction
	# so it is a list of length N with sublists
	# of size k
	l_diagrams = make_diagrams(scaled_objects, matrix_dir)
	dist_mat = np.zeros((N,N))
	list_inputs = [(i, j, k, l_diagrams) for i in range(N)
										 for j in range(i+1, N)]
	with mp.Pool(processes=workers) as pool:
		distances = pool.starmap(calculate_distance_scale, list_inputs)
	for tup in distances:
		dist_mat[tup[0],tup[1]] = tup[2]
	dist_mat += dist_mat.T
	return dist_mat

Algorithms
---
One algorithm used is the Hungarian (or Munkres) algorithm.  The alogirthm is used in situations where assignments with an associated cost must be made and the goal is to select the assignment to minimize the cost.  Here we use this algorithm to calculate the distance between persistence diagrams. The distance between persistence diagrams is the sum of the distances between the points of the first persistence diagram paired with the points of the second diagram and additional points on the diagonal.  Selecting the pairing that minimzies this distance can be achieved using the Munkres algorithm.

Another algorithm used in our code is the Union-Find algorithm.  This algorithm is used in the construction of the persistence diagrams. During the construction we must keep track of when disjoint components merge.  We view each component as a tree.  When two components merge we join the roots of the trees.  This allows us to find when disjoint components merge.

Section 4:  Tests
---
Write some tests. In particular, compare results from our codes to results in the paper to ensure that our codes yield the same results.  Test on simple simulated data.

Section 5:  Optimization
---
#### Part 1: compilation optimization
We tried optimizing the performance of the codes using `numba` just-in-time (JIT) compilation, `Cython`, embarrasingly parallel processing. Based on the input file `Class1_Sample1.mat`, most functions perform very well, taking only miliseconds to process such a shape with $375$ vertices. Assuming we want to measure the distance between such a shape to another shape with $375$ vertices, it takes $33.6s$ for the current Munkres algorithm to finish running, suggesting room for improvement.

We used the function `linear_sum_assignment` (Source: https://github.com/scipy/scipy/blob/master/scipy/optimize/_hungarian.py#L13-L107`scipy) from `scipy.optimize`. To improve its performance, we tried numba JIT computation, simple compilation in `Cython`, cythonizing via static typing and `cnumpy` iteration and wrapping `C` codes. However, no correct and easily accessible `C` or `C++` functions have been found. Neither have we achieve significant improvement in `Python`. Specifically, the major problem in cythonizing concerns the fast array declarations in `cdef` classes. Fast array declarations, however, are currently not accessible in the fields of cdef classes or as global variables, according to Cython documentation
(ref: http://cython.readthedocs.io/en/latest/src/tutorial/numpy.html).

We benchmarked the different optimization strategies based on a $1,000 \times 1,000$ cost matrix. The performances are  summarised in the table below:

|  | original function | numba JIT  | Simple Cython compilation | Cythonizing via static typing & `cNumPy` |
|-----------|-------------------|------------|---------------------------|------------------------------------------|
| Wall time | 1 min 41 s        | 1 min 27 s | 1 min 46 s                | 1 min 42 s                               |


As can be seen from the above table, no consistent outperformance has been achieved. 

#### Part 2: algorithm optimization

We speeded up the function `make_diagram` via better algorithm. 